# Обучение модели временного ряда

По 3м точкам финансовых показателей пытаемся предсказать четвертую точку на год вперед. Затем смотрим, как изменение финансовых показателей повлияло на проставление метки дефолта.

Например, модель Prophet принимает на вход датафрейм из 2х колонок: дата и значение показателя. Таким образом, наша модель обучится только на одной компании и сможет предсказывать значение какого-то параметра на год вперед. Так мы для каждой компании сможем без обучающей выборки и без обобщения по другим компаниям определить следующее значение этого параметра, а затем рассчитать метку дефолта.

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('/content/drive/Shared drives/Кредитные риски')

In [0]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import accuracy_score, confusion_matrix
from fbprophet import Prophet
from tqdm import tqdm_notebook

In [0]:
DIR_IN = 'Датасеты/revision_007/'

## Считывание данных (только некоторые коды, необходимые для рассчета дефолта)

In [0]:
df_train = pd.read_pickle(f'{DIR_IN}companies_ready_train.pkl')
y_train = df_train['target']
x_train = df_train.drop(columns=['target'])

df_test = pd.read_pickle(f'{DIR_IN}companies_ready_test.pkl')
y_test = df_test['target']
x_test = df_test.drop(columns=['target'])

df_prod = pd.read_pickle(f'{DIR_IN}companies_ready_prod.pkl')
y_prod = df_prod['target']
x_prod = df_prod.drop(columns=['target'])

## Обучение модели для каждой компании

Расчет дефолта для каждой компании:

```
code_12003 != 0 and code_15003 != 0 and code_12003 / code_15003 < 0.5

code_16004 / code_16003 > 2

code_12303 != 0 and code_12304 != 0 and (code_12303 / code_16003) / (code_12004 / code_16004) > 1

code_12103 != 0 and code_12104 != 0 and (code_12103 / code_12104 > 3 or code_12104 / code_12103 > 3)
```




In [0]:
code = ['1200', '1210', '1500', '1230', '1600']

In [0]:
def is_default(x, y):
    '''
    Расчитывает дефолт компании
    False(1-2 poins get) - ok
    True(3-5 points get) - дефолт
    '''
    cond_a = ((y['1200'] != 0) & (y['1500'] != 0) & (y['1200']/y['1500'] < 0.5)).astype(np.uint8)
    cond_b = (x['year_0_1600']/y['1600'] > 2).astype(np.uint8)
    cond_c = ((y['1230'] != 0) & (x['year_0_1230'] != 0) & ((y['1230']/y['1600'])/(x['year_0_1200']/x['year_0_1600']) > 1)).astype(np.uint8)
    cond_d = ((y['1210'] != 0) & (x['year_0_1210'] !=0 ) & ((y['1210']/x['year_0_1210'] > 3) | (x['year_0_1210']/y['1210'] > 3))).astype(np.uint8)

    return (cond_a + cond_b + cond_c + cond_d + 1) > 2

In [0]:
y_pred = []
for index, row in tqdm_notebook(x_test[:1000].iterrows(), total=1000):
    answer = {}
    for c in code:
        data = row.loc[[f'year_-2_{c}', f'year_-1_{c}', f'year_0_{c}']]
        df = pd.DataFrame({'ds': ['2016-01-01', '2017-01-01', '2018-01-01'], 'y': data})
        prph = Prophet()
        prph.fit(df)
        future = prph.make_future_dataframe(periods=1, freq='Y')
        forecast = prph.predict(future)
        answer[c] = forecast.iloc[3, 1]
    y = pd.DataFrame([answer])
    y_pred.append(is_default(row, y).astype(np.uint8).values[0])

In [0]:
with open(f'{DIR_IN}/prophet_predict.pkl', 'wb') as f:
    pickle.dump(np.array(y_pred), f)

In [0]:
accuracy_score(y_pred, y_test[:1000])

0.656

In [0]:
confusion_matrix(y_pred, y_test[:1000])

array([[640, 136],
       [208,  16]])

In [0]:
sum(y_test[:1000])

152

Модель на временных рядах верно определила всего 16 дефолтных компаний из 152 дефолтных. При этом 208 хороших компаний причислила к классу дефолтных, хотя они таковыми не являлись.

Такая модель не выдает вероятности принадлежности к классу дефолта, что также является ее недостатком.